In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_json('../dataset/sstubs')

In [3]:
df.shape

(10231, 14)

### Overview

In [4]:
df.sample(5)

,bugType,fixCommitSHA1,fixCommitParentSHA1,bugFilePath,fixPatch,projectName,bugLineNum,bugNodeStartChar,bugNodeLength,fixLineNum,fixNodeStartChar,fixNodeLength,sourceBeforeFix,sourceAfterFix
3064,DIFFERENT_METHOD_SAME_ARGS,67685860541cf7f66d048f314e1b6084591974c3,7f417056d2f3bf4b0f1a839922c68829f84a170f,camel-core/src/test/java/org/apache/camel/comp...,diff --git a/camel-core/src/test/java/org/apac...,apache.camel,135,5309,55,135,5376,65,"result.expectedBodiesReceived(""A+C+E+G+I"",""B+D...","result.expectedBodiesReceivedInAnyOrder(""A+C+E..."
3096,CHANGE_IDENTIFIER,bafb1f877934f826d5f74f9f33e824dd1aa30432,fdfbb0ca84342d5b4f328e1f9a3bfb2058552145,camel-core/src/main/java/org/apache/camel/conv...,diff --git a/camel-core/src/main/java/org/apac...,apache.camel,88,3201,10,88,3201,11,@Converter,@Deprecated
5821,CHANGE_IDENTIFIER,23423085f9af64ba33a2031298f46474317cb425,daa13a46ab8f1895a92975fe1d042ef86a340a76,src/com/google/javascript/jscomp/DefinitionUse...,diff --git a/src/com/google/javascript/jscomp/...,google.closure-compiler,172,5765,46,172,5765,45,"this.nameDefinitionMultimap.remove(name,node)","this.nameDefinitionMultimap.remove(name,def)"
8687,CHANGE_IDENTIFIER,e2854295f7c484671ce69bff7ec37fcb0c413a32,c2c976107dfcc44fbd6bc2af8435b2c440ca592d,src/main/java/spark/ExceptionHandlerImpl.java,diff --git a/src/main/java/spark/ExceptionHand...,perwendel.spark,59,1965,19,59,1965,11,Exception exception,T exception
2486,DIFFERENT_METHOD_SAME_ARGS,e5298f62da9c8ff2319709bdd35aceef75e97228,94af4251adf9d306bfead934c7587492ddcee6f9,components/camel-jmx/src/test/java/org/apache/...,diff --git a/components/camel-jmx/src/test/jav...,apache.camel,62,1895,27,62,1895,28,"getRegistry().put(""hb"",hb)","getRegistry().bind(""hb"",hb)"


### Sample Entry

In [5]:
for k in df.iloc[0].keys():
    print(('**{}**'.format(k)))
    print(df.iloc[3][k])
    print()

**bugType**
DIFFERENT_METHOD_SAME_ARGS

**fixCommitSHA1**
114870164ba9822afc3a92eefe5258f54a1c6a17

**fixCommitParentSHA1**
845ab7d135ab688e4943bd0f5472f0a5401528f8

**bugFilePath**
modules/activiti-engine/src/test/java/org/activiti/engine/test/api/runtime/TaskInvolvementTest.java

**fixPatch**
diff --git a/modules/activiti-engine/src/test/java/org/activiti/engine/test/api/runtime/TaskInvolvementTest.java b/modules/activiti-engine/src/test/java/org/activiti/engine/test/api/runtime/TaskInvolvementTest.java
index 4de3611..f71b5a3 100644
--- a/modules/activiti-engine/src/test/java/org/activiti/engine/test/api/runtime/TaskInvolvementTest.java
+++ b/modules/activiti-engine/src/test/java/org/activiti/engine/test/api/runtime/TaskInvolvementTest.java
@@ -26,7 +26,7 @@
             assertEquals(1, taskService.createTaskQuery()
                     .or()
                     .taskInvolvedUser("involvedUser")
-                    .taskInvolvedGroups(groups)
+                    .taskInvolvedGroup

### Identify Bug Types

In [46]:
bugTypes = df.groupby('bugType').agg(
    count=pd.NamedAgg(column='bugType', aggfunc='count'),
    sourceBeforeFix=pd.NamedAgg(column='sourceBeforeFix', aggfunc='last'),
    sourceAfterFix=pd.NamedAgg(column='sourceAfterFix', aggfunc='last'),
)
bugTypes.index.name = None
percentage = [round(row / df.shape[0] * 100, 2) for row in bugTypes['count']]
bugTypes.insert(1, 'percentage', percentage)

In [47]:
bugTypes

,count,percentage,sourceBeforeFix,sourceAfterFix
ADD_THROWS_EXCEPTION,68,0.66,0,0
CHANGE_CALLER_IN_FUNCTION_CALL,187,1.83,userStreamListener.onStatus(new StatusJSONImpl...,listener.onStatus(new StatusJSONImpl(json))
CHANGE_IDENTIFIER,3265,31.91,assertTrue(found),assertFalse(found)
CHANGE_MODIFIER,1852,18.10,138,202
CHANGE_NUMERAL,1137,11.11,args.length < 1,args.length < 2
CHANGE_OPERAND,120,1.17,preDestroys != null,prePassivates != null
CHANGE_OPERATOR,275,2.69,responseCode < OK && MULTIPLE_CHOICES <= respo...,responseCode < OK || MULTIPLE_CHOICES <= respo...
CHANGE_UNARY_OPERATOR,170,1.66,JedisClusterHashTagUtil.isClusterCompliantMatc...,!JedisClusterHashTagUtil.isClusterCompliantMat...
DELETE_THROWS_EXCEPTION,48,0.47,1,1
DIFFERENT_METHOD_SAME_ARGS,1486,14.52,assertTrue(found),assertFalse(found)


### Explore Each Bug Type Individually

#### 0. ADD_THROWS_EXCEPTION

In [198]:
bugTypeDf = df[df.bugType == 'ADD_THROWS_EXCEPTION']
bugTypeSample = bugTypeDf.iloc[0]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

1
1

diff --git a/core/client/fs/src/test/java/alluxio/client/file/policy/LocalFirstAvoidEvictionPolicyTest.java b/core/client/fs/src/test/java/alluxio/client/file/policy/LocalFirstAvoidEvictionPolicyTest.java
index 9bb1562..b10143e 100644
--- a/core/client/fs/src/test/java/alluxio/client/file/policy/LocalFirstAvoidEvictionPolicyTest.java
+++ b/core/client/fs/src/test/java/alluxio/client/file/policy/LocalFirstAvoidEvictionPolicyTest.java
@@ -34,7 +34,7 @@
 public class LocalFirstAvoidEvictionPolicyTest {
 
   @Test
-  public void chooseClosestTierAvoidEviction() {
+  public void chooseClosestTierAvoidEviction() throws Exception {
     List<BlockWorkerInfo> workers = new ArrayList<>();
     workers.add(worker(Constants.GB, Constants.MB, "node2", "rack3"));
     workers.add(worker(Constants.GB, 0, "node3", "rack2"));



In [199]:
print(np.unique(bugTypeDf.sourceBeforeFix))
print(np.unique(bugTypeDf.sourceAfterFix))

['0' '1' '2' '33' '34' '4' '9']
['0' '1' '2' '33' '34' '4' '9']


- Simply contains diffs that include a `throws Exception`
- Values in `sourceBeforeFix` or `sourceAfterFix` ain't clear.
  Need to find corresponding docs.

#### 1. CHANGE_CALLER_IN_FUNCTION_CALL

In [5]:
bugTypeDf = df[df.bugType == 'CHANGE_CALLER_IN_FUNCTION_CALL']
bugTypeSample = bugTypeDf.iloc[0]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)
print(bugTypeSample.fixCommitSHA1)

DATABASE_TYPE_MYSQL.equals(databaseType)
DATABASE_TYPE_MSSQL.equals(databaseType)

diff --git a/modules/activiti-engine/src/main/java/org/activiti/engine/impl/cfg/ProcessEngineConfigurationImpl.java b/modules/activiti-engine/src/main/java/org/activiti/engine/impl/cfg/ProcessEngineConfigurationImpl.java
index 6335125..7604803 100755
--- a/modules/activiti-engine/src/main/java/org/activiti/engine/impl/cfg/ProcessEngineConfigurationImpl.java
+++ b/modules/activiti-engine/src/main/java/org/activiti/engine/impl/cfg/ProcessEngineConfigurationImpl.java
@@ -1209,7 +1209,7 @@
 
       // Special care for MSSQL, as it has a hard limit of 2000 params per statement (incl bulk statement).
       // Especially with executions, with 100 as default, this limit is passed.
-      if (DATABASE_TYPE_MYSQL.equals(databaseType)) {
+      if (DATABASE_TYPE_MSSQL.equals(databaseType)) {
         maxNrOfStatementsInBulkInsert = DEFAULT_MAX_NR_OF_STATEMENTS_BULK_INSERT_SQL_SERVER;
       }
       

9b649ac6da42

#### 2. CHANGE_IDENTIFIER

In [6]:
bugTypeDf = df[df.bugType == 'CHANGE_IDENTIFIER']
bugTypeSample = bugTypeDf.iloc[2]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

DATABASE_TYPE_MYSQL.equals(databaseType)
DATABASE_TYPE_MSSQL.equals(databaseType)

diff --git a/modules/activiti-engine/src/main/java/org/activiti/engine/impl/cfg/ProcessEngineConfigurationImpl.java b/modules/activiti-engine/src/main/java/org/activiti/engine/impl/cfg/ProcessEngineConfigurationImpl.java
index 6335125..7604803 100755
--- a/modules/activiti-engine/src/main/java/org/activiti/engine/impl/cfg/ProcessEngineConfigurationImpl.java
+++ b/modules/activiti-engine/src/main/java/org/activiti/engine/impl/cfg/ProcessEngineConfigurationImpl.java
@@ -1209,7 +1209,7 @@
 
       // Special care for MSSQL, as it has a hard limit of 2000 params per statement (incl bulk statement).
       // Especially with executions, with 100 as default, this limit is passed.
-      if (DATABASE_TYPE_MYSQL.equals(databaseType)) {
+      if (DATABASE_TYPE_MSSQL.equals(databaseType)) {
         maxNrOfStatementsInBulkInsert = DEFAULT_MAX_NR_OF_STATEMENTS_BULK_INSERT_SQL_SERVER;
       }
       



- Includes any type of identifier change ranging from object, method, variable and so on
- A single diff can be present in multiple `bugType`s. Entry with `fixCommitSHA1`:
  `"9b649ac6da4293d736676de895a67aaf87fc5979"` is present in both `CHANGE_CALLER_IN_FUNCTION_CALL`
  and `CHANGE_IDENTIFIER`.

#### 3. CHANGE_MODIFIER

In [220]:
bugTypeDf = df[df.bugType == 'CHANGE_MODIFIER']
bugTypeSample = bugTypeDf.iloc[3]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

2
18

diff --git a/core/server/worker/src/main/java/alluxio/SessionInfo.java b/core/server/worker/src/main/java/alluxio/SessionInfo.java
index 51a2fa5..11432aa 100644
--- a/core/server/worker/src/main/java/alluxio/SessionInfo.java
+++ b/core/server/worker/src/main/java/alluxio/SessionInfo.java
@@ -24,7 +24,7 @@
   private final long mSessionId;
 
   private long mLastHeartbeatMs;
-  private int mSessionTimeoutMs;
+  private final int mSessionTimeoutMs;
 
   /**
    * Creates a new instance of {@link SessionInfo}.



In [221]:
print(np.unique(bugTypeDf.sourceBeforeFix))
print(np.unique(bugTypeDf.sourceAfterFix))

['0' '1' '10' '1024' '1025' '1034' '12' '128' '138' '146' '16' '17' '18'
 '2' '20' '24' '25' '26' '28' '32' '33' '34' '4' '49' '66' '8' '9']
['0' '1' '10' '1025' '1028' '1033' '12' '130' '146' '154' '16' '17' '18'
 '192' '2' '20' '202' '24' '25' '26' '28' '32' '33' '34' '36' '4' '40'
 '41' '42' '64' '66' '68' '74' '76' '8' '9']


- Contains changes in modifiers like `public`, `private`, `abstract`, `final` etc.
- Not sure the values of `sourceBeforeFix` and `sourceAfterFix`.

#### 4. CHANGE_NUMERAL

In [202]:
bugTypeDf = df[df.bugType == 'CHANGE_NUMERAL']
bugTypeSample = bugTypeDf.iloc[1]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

waitForJobExecutorToProcessAllJobs(3000,500)
waitForJobExecutorToProcessAllJobs(5000,500)

diff --git a/modules/activiti-camel/src/test/java/org/activiti/camel/revisited/AsyncProcessRevisitedTest.java b/modules/activiti-camel/src/test/java/org/activiti/camel/revisited/AsyncProcessRevisitedTest.java
index ec34354..951cf7a 100755
--- a/modules/activiti-camel/src/test/java/org/activiti/camel/revisited/AsyncProcessRevisitedTest.java
+++ b/modules/activiti-camel/src/test/java/org/activiti/camel/revisited/AsyncProcessRevisitedTest.java
@@ -53,7 +53,7 @@
     ProcessInstance processInstance = runtimeService.startProcessInstanceByKey("asyncCamelProcessRevisited");
     List<Execution> executionList = runtimeService.createExecutionQuery().list();
     assertEquals(3, executionList.size());
-    waitForJobExecutorToProcessAllJobs(3000, 500);
+    waitForJobExecutorToProcessAllJobs(5000, 500);
     assertEquals(0, runtimeService.createProcessInstanceQuery().processInstanceId(processInstance.getId

- Contains changing one number to another
- Might change the type

#### 5. CHANGE_OPERAND

In [203]:
bugTypeDf = df[df.bugType == 'CHANGE_OPERAND']
bugTypeSample = bugTypeDf.iloc[28]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

"" + SYSTEM_PROPERTIES_MODE_FALLBACK
"" + ENVIRONMENT_VARIABLES_MODE_OVERRIDE

diff --git a/components/camel-properties/src/main/java/org/apache/camel/component/properties/PropertiesComponent.java b/components/camel-properties/src/main/java/org/apache/camel/component/properties/PropertiesComponent.java
index b6a2df5..5094e47 100644
--- a/components/camel-properties/src/main/java/org/apache/camel/component/properties/PropertiesComponent.java
+++ b/components/camel-properties/src/main/java/org/apache/camel/component/properties/PropertiesComponent.java
@@ -120,7 +120,7 @@
     private Properties overrideProperties;
     @Metadata(defaultValue = "" + SYSTEM_PROPERTIES_MODE_OVERRIDE, enums = "0,1,2")
     private int systemPropertiesMode = SYSTEM_PROPERTIES_MODE_OVERRIDE;
-    @Metadata(defaultValue = "" + SYSTEM_PROPERTIES_MODE_FALLBACK, enums = "0,1,2")
+    @Metadata(defaultValue = "" + ENVIRONMENT_VARIABLES_MODE_OVERRIDE, enums = "0,1,2")
     private int environmentVariableMode = ENVIR

- Contains changing an operand of an operator
- Need to check if contains changing both operator
- Need to check if contains `=` operator

#### 6. CHANGE_OPERATOR

In [204]:
bugTypeDf = df[df.bugType == 'CHANGE_OPERATOR']
bugTypeSample = bugTypeDf.iloc[28]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

i > BY_WOOL_DATA.length
i >= BY_WOOL_DATA.length

diff --git a/src/main/java/org/bukkit/DyeColor.java b/src/main/java/org/bukkit/DyeColor.java
index b8e59a4..58f517b 100644
--- a/src/main/java/org/bukkit/DyeColor.java
+++ b/src/main/java/org/bukkit/DyeColor.java
@@ -164,7 +164,7 @@
      */
     public static DyeColor getByWoolData(final byte data) {
         int i = 0xff & data;
-        if (i > BY_WOOL_DATA.length) {
+        if (i >= BY_WOOL_DATA.length) {
             return null;
         }
         return BY_WOOL_DATA[i];
@@ -179,7 +179,7 @@
      */
     public static DyeColor getByDyeData(final byte data) {
         int i = 0xff & data;
-        if (i > BY_DYE_DATA.length) {
+        if (i >= BY_DYE_DATA.length) {
             return null;
         }
         return BY_DYE_DATA[i];



#### 7. CHANGE_UNARY_OPERATOR

In [205]:
bugTypeDf = df[df.bugType == 'CHANGE_UNARY_OPERATOR']
bugTypeSample = bugTypeDf.iloc[5]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

ufsDeleter.delete(alluxioUriToDel,delInode)
!ufsDeleter.delete(alluxioUriToDel,delInode)

diff --git a/core/server/master/src/main/java/alluxio/master/file/DefaultFileSystemMaster.java b/core/server/master/src/main/java/alluxio/master/file/DefaultFileSystemMaster.java
index e015ad9..9def980 100644
--- a/core/server/master/src/main/java/alluxio/master/file/DefaultFileSystemMaster.java
+++ b/core/server/master/src/main/java/alluxio/master/file/DefaultFileSystemMaster.java
@@ -1207,7 +1207,7 @@
               unmountInternal(alluxioUriToDel);
             } else if (!deleteOptions.isAlluxioOnly()) {
               // Attempt to delete node if all children were deleted successfully
-              failedToDelete = ufsDeleter.delete(alluxioUriToDel, delInode);
+              failedToDelete = !ufsDeleter.delete(alluxioUriToDel, delInode);
             }
           } catch (InvalidPathException e) {
             LOG.warn(e.getMessage());



- Need to check if contains anything other than `!`

#### 8. DELETE_THROWS_EXCEPTION

In [206]:
bugTypeDf = df[df.bugType == 'DELETE_THROWS_EXCEPTION']
bugTypeSample = bugTypeDf.iloc[5]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

1
1

diff --git a/src/main/java/com/alibaba/druid/support/spring/stat/BeanTypeAutoProxyCreator.java b/src/main/java/com/alibaba/druid/support/spring/stat/BeanTypeAutoProxyCreator.java
index 42409d8..6ce7f98 100644
--- a/src/main/java/com/alibaba/druid/support/spring/stat/BeanTypeAutoProxyCreator.java
+++ b/src/main/java/com/alibaba/druid/support/spring/stat/BeanTypeAutoProxyCreator.java
@@ -52,7 +52,7 @@
         this.targetBeanType = targetClass;
     }
 
-    public void setApplicationContext(ApplicationContext context) throws BeansException {
+    public void setApplicationContext(ApplicationContext context) {
         this.context = context;
     }
 



In [207]:
print(np.unique(bugTypeDf.sourceBeforeFix))
print(np.unique(bugTypeDf.sourceAfterFix))

['0' '1' '10' '4' '9']
['0' '1' '10' '4' '9']


- Values in `sourceBeforeFix` or `sourceAfterFix` are different from `ADD_THROWS_EXCEPTION`.
  Need to explore outersecting types (`'2' '10' '33' '34')`.

#### 9. DIFFERENT_METHOD_SAME_ARGS

In [208]:
bugTypeDf = df[df.bugType == 'DIFFERENT_METHOD_SAME_ARGS']
bugTypeSample = bugTypeDf.iloc[8]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

xtw.writeCharacters(customProperty.getSimpleValue())
xtw.writeCData(customProperty.getSimpleValue())

diff --git a/modules/activiti-bpmn-converter/src/main/java/org/activiti/bpmn/converter/UserTaskXMLConverter.java b/modules/activiti-bpmn-converter/src/main/java/org/activiti/bpmn/converter/UserTaskXMLConverter.java
index d588c3d..fb523c6 100644
--- a/modules/activiti-bpmn-converter/src/main/java/org/activiti/bpmn/converter/UserTaskXMLConverter.java
+++ b/modules/activiti-bpmn-converter/src/main/java/org/activiti/bpmn/converter/UserTaskXMLConverter.java
@@ -162,7 +162,7 @@
           didWriteExtensionStartElement = true;
         }
         xtw.writeStartElement(ACTIVITI_EXTENSIONS_PREFIX, customProperty.getName(), ACTIVITI_EXTENSIONS_NAMESPACE);
-        xtw.writeCharacters(customProperty.getSimpleValue());
+        xtw.writeCData(customProperty.getSimpleValue());
         xtw.writeEndElement();
       }
     }



- Need to check if shares entries with `CHANGE_IDENTIFIER`

#### 10. LESS_SPECIFIC_IF

In [209]:
bugTypeDf = df[df.bugType == 'LESS_SPECIFIC_IF']
bugTypeSample = bugTypeDf.iloc[4]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

!ufsSyncChecker.isDirectoryInSync(parentUri)
alluxioUriToDel.equals(inodePath.getUri()) || !ufsSyncChecker.isDirectoryInSync(parentUri)

diff --git a/core/server/master/src/main/java/alluxio/master/file/DefaultFileSystemMaster.java b/core/server/master/src/main/java/alluxio/master/file/DefaultFileSystemMaster.java
index 435021b..77a8b9f 100644
--- a/core/server/master/src/main/java/alluxio/master/file/DefaultFileSystemMaster.java
+++ b/core/server/master/src/main/java/alluxio/master/file/DefaultFileSystemMaster.java
@@ -1226,7 +1226,8 @@
               String ufsUri = resolution.getUri().toString();
               UnderFileSystem ufs = resolution.getUfs();
               AlluxioURI parentUri = alluxioUriToDel.getParent();
-              if (!ufsSyncChecker.isDirectoryInSync(parentUri)) {
+              if (alluxioUriToDel.equals(inodePath.getUri())
+                  || !ufsSyncChecker.isDirectoryInSync(parentUri)) {
                 // Parent will not recursively delete, so delete thi

- Contains `if`s where more condition added with `OR`

#### 11. MORE_SPECIFIC_IF

In [210]:
bugTypeDf = df[df.bugType == 'MORE_SPECIFIC_IF']
bugTypeSample = bugTypeDf.iloc[4]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

idAnnotation != null
idAnnotation != null && !method.isBridge()

diff --git a/modules/activiti-engine/src/main/java/org/activiti/engine/impl/variable/JPAEntityScanner.java b/modules/activiti-engine/src/main/java/org/activiti/engine/impl/variable/JPAEntityScanner.java
index 5fbbb87..f3e9bf7 100644
--- a/modules/activiti-engine/src/main/java/org/activiti/engine/impl/variable/JPAEntityScanner.java
+++ b/modules/activiti-engine/src/main/java/org/activiti/engine/impl/variable/JPAEntityScanner.java
@@ -70,7 +70,7 @@
     Id idAnnotation = null;
     for (Method method : methods) {
       idAnnotation = method.getAnnotation(Id.class);
-      if (idAnnotation != null) {
+      if(idAnnotation != null && !method.isBridge()) {
         idMethod = method;
         break;
       }



- Contains `if`s where more condition added with `AND`

#### 12. OVERLOAD_METHOD_DELETED_ARGS

In [211]:
bugTypeDf = df[df.bugType == 'OVERLOAD_METHOD_DELETED_ARGS']
bugTypeSample = bugTypeDf.iloc[4]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

new LocalTachyonClusterResource(Constants.GB,Constants.KB,BLOCK_SIZE,Constants.KEY_VALUE_ENABLED,"true",Constants.KEY_VALUE_PARTITION_SIZE_BYTES_MAX,Integer.toString(KEY_VALUE_PARTITION_SIZE))
new LocalTachyonClusterResource(Constants.GB,BLOCK_SIZE,Constants.KEY_VALUE_ENABLED,"true",Constants.KEY_VALUE_PARTITION_SIZE_BYTES_MAX,Integer.toString(KEY_VALUE_PARTITION_SIZE))

diff --git a/tests/src/test/java/tachyon/client/keyvalue/hadoop/KeyValueStoreMapReduceIntegrationTest.java b/tests/src/test/java/tachyon/client/keyvalue/hadoop/KeyValueStoreMapReduceIntegrationTest.java
index 9bd8778..b34b8aa 100644
--- a/tests/src/test/java/tachyon/client/keyvalue/hadoop/KeyValueStoreMapReduceIntegrationTest.java
+++ b/tests/src/test/java/tachyon/client/keyvalue/hadoop/KeyValueStoreMapReduceIntegrationTest.java
@@ -50,7 +50,7 @@
 
   @ClassRule
   public static LocalTachyonClusterResource sLocalTachyonClusterResource =
-      new LocalTachyonClusterResource(Constants.GB, Constants.KB, BLOCK_SIZE,
+   

- Contains if an argument is removed from method

#### 13. OVERLOAD_METHOD_MORE_ARGS

In [212]:
bugTypeDf = df[df.bugType == 'OVERLOAD_METHOD_MORE_ARGS']
bugTypeSample = bugTypeDf.iloc[4]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

IOUtils.toString(stencilsetStream)
IOUtils.toString(stencilsetStream,"utf-8")

diff --git a/modules/activiti-modeler/src/main/java/org/activiti/rest/editor/main/StencilsetRestResource.java b/modules/activiti-modeler/src/main/java/org/activiti/rest/editor/main/StencilsetRestResource.java
index b07c4a7..4502b19 100644
--- a/modules/activiti-modeler/src/main/java/org/activiti/rest/editor/main/StencilsetRestResource.java
+++ b/modules/activiti-modeler/src/main/java/org/activiti/rest/editor/main/StencilsetRestResource.java
@@ -27,11 +27,11 @@
 @RestController
 public class StencilsetRestResource {
   
-  @RequestMapping(value="/editor/stencilset", method = RequestMethod.GET, produces = "application/json")
+  @RequestMapping(value="/editor/stencilset", method = RequestMethod.GET, produces = "application/json;charset=utf-8")
   public @ResponseBody String getStencilset() {
     InputStream stencilsetStream = this.getClass().getClassLoader().getResourceAsStream("stencilset.json");
     try {
-

- Contains if an argument is added to method
- Need to check if more than one argument are added anywhere

#### 14. SWAP_ARGUMENTS

In [213]:
bugTypeDf = df[df.bugType == 'SWAP_ARGUMENTS']
bugTypeSample = bugTypeDf.iloc[0]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

new NetAddress(resolvedHost,resolvedPort,-1)
new NetAddress(resolvedHost,-1,resolvedPort)

diff --git a/servers/src/main/java/tachyon/master/BlockInfo.java b/servers/src/main/java/tachyon/master/BlockInfo.java
index 10f3b21..ec659db 100644
--- a/servers/src/main/java/tachyon/master/BlockInfo.java
+++ b/servers/src/main/java/tachyon/master/BlockInfo.java
@@ -187,7 +187,8 @@
           } catch (NumberFormatException nfe) {
             continue;
           }
-          ret.add(new NetAddress(resolvedHost, resolvedPort, -1));
+          // The resolved port is the data transfer port not the rpc port
+          ret.add(new NetAddress(resolvedHost, -1, resolvedPort));
         }
       }
     }



- Very few entries here are actually bugs. Most are assertions.
- As expected, where it is really error, the types of both the arguments are the same.

#### 15. SWAP_BOOLEAN_LITERAL

In [214]:
bugTypeDf = df[df.bugType == 'SWAP_BOOLEAN_LITERAL']
bugTypeSample = bugTypeDf.iloc[9]
print(bugTypeSample.sourceBeforeFix)
print(bugTypeSample.sourceAfterFix)
print()
print(bugTypeSample.fixPatch)

setAttributeInternal(tempInodePath,true,opTimeMs,options)
setAttributeInternal(tempInodePath,false,opTimeMs,options)

diff --git a/core/server/src/main/java/alluxio/master/file/FileSystemMaster.java b/core/server/src/main/java/alluxio/master/file/FileSystemMaster.java
index 3e6c8a8..79939ab 100644
--- a/core/server/src/main/java/alluxio/master/file/FileSystemMaster.java
+++ b/core/server/src/main/java/alluxio/master/file/FileSystemMaster.java
@@ -2294,14 +2294,14 @@
         for (Inode<?> inode : inodeChildren) {
           // the path to inode for getPath should already be locked.
           tempInodePath.setDescendant(inode, mInodeTree.getPath(inode));
-          List<Inode<?>> persistedInodes = setAttributeInternal(tempInodePath, true, opTimeMs,
+          List<Inode<?>> persistedInodes = setAttributeInternal(tempInodePath, false, opTimeMs,
               options);
           journalPersistedInodes(persistedInodes);
           journalSetAttribute(tempInodePath, opTimeMs, options);
 

- Contains entries where `true` is changed to `false` or vice-versa.

### Key Notes

* If there are multiple diffs, then they are counted as different entries
